# JupyThree

Efficient massive pointclouds and easy triangular meshes

## Pointclouds

### Basic usage

In [ ]:
import numpy as np
from jupythree.pointcloud import pointcloud

Massive pointcloud (2 millions points)

In [ ]:
import numpy as np
from jupythree.pointcloud import pointcloud

N = 2000000
x = np.random.random((N,3))/2 - 0.25 # cube

In [ ]:
pointcloud(x).show(title="2 Million points")

> From now on, the other examples will only have 50k points

Apply a color to every point

In [ ]:
N = 50000
x = np.random.random((N,3))
c = x.copy()
x = x/2 - 0.25

pointcloud(x, color=c).show(title="RGB <-> position", width=800, height=600)

Get a slider to control the radius of the points

In [ ]:
pointcloud(x, color=c).with_slider().show()

### Scalar fields with colormaps from `matplotlib`

This cell also demonstrates how to show several windows in a row, synchronizing the rotations

> **Note**: when using the argument `master=...`, only the last window is rendered at first, you need to click somewhere to display the other ones 

In [ ]:
# color corresponding to Y coordinate

from ipywidgets import HBox
from matplotlib import cm

wa = pointcloud(x, color=x[:,1]               ).to_window(title="viridis (default)"                       )
wb = pointcloud(x, color=x[:,1], cmap=cm.jet  ).to_window(title="jet - invisible until clicked", master=wa)
wc = pointcloud(x, color=x[:,1], cmap=cm.ocean).to_window(title="ocean"                        , master=wa)

HBox([
    wa.show(),
    wb.show(),
    wc.show()
])

### Combining pointclouds in the same window

In [ ]:
y = np.random.random((5000,3))/2 - 0.25
y /= 2*np.linalg.norm(y, axis=1, keepdims=True) # project to ball
c2 = y[:, 0]

In [ ]:
from jupythree.window import window

window(pointcloud(x), pointcloud(y, c2, radius=1)).show()

### Easy animations

You can update the properties of a pointcloud window (positions and/or colors) using `.update()` method:

```python
pc = pointcloud(X1)
X2 = ...
pc.update(pc=X2)
```

> _Tip_ : this can be put every N steps of an optimization procedure (gradient descent, neural network training, etc.) to visualize intermediate states

In [ ]:
# 1. Setup
N = 10000

X = (np.random.random((N,3)) - 0.5)/2
color = X[:,0]
pc = pointcloud(X, color)

pc.show(title="This window will animate when next cell is executed")

In [ ]:
# 2. Animate (apply random perturbations)
import time

sigma = 0.002
n_seconds = 3
fps = 30

tick = 1./fps
for i in range(n_seconds * fps):
    X += np.random.randn(N,3) * sigma
    pc.update(pc=X, color=color) # if color is not specified, resets to default color (red)
    time.sleep(tick)

## Support for meshes

The `mesh` class behaves like `pointcloud`, with the differences being:

1. It requires both vertices `V` and faces `F`, both are arrays (see example below)
2. You can either assign a `constant_color` in rgb hex format (like `u"#00aaff"` for aqua blue), or `vertex_color` which will then expect as many colors as vertices in `V` (either scalars with a colormap, or [r,g,b] triplets in [0,1])

### Basic usage

In [ ]:
from jupythree.mesh import mesh

In [ ]:
# load the demo mesh

V = np.loadtxt("example_mesh/V.txt")
F = np.loadtxt("example_mesh/F.txt")
print(f"Array shapes: {V.shape=}, {F.shape=}")
print(f"Values range: {V.min()=}, {V.max()=}")

In [ ]:
# Simple way
mesh(V, F, constant_color=u"#00aaff").show(title="Here in aqua (default is gray)")

### Vertex colors

Color according to X axis + show red wireframe

In [ ]:
mesh(V, F, vertex_color=V[:,0], lineopacity=1., linecolor=u"#ff0000").show(
    title="Colored along X axis"
)

### Combined with a pointcloud

In [ ]:
window(mesh(V, F), pointcloud(y, c2, radius=1)).show()